# [ 은닉계층 1개로 구성된 다층 신경망 ]
### 설명을 위해 입력을 입력층, 출력을 출력층이라 서술하였습니다.

In [13]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

In [14]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.5

In [15]:
# 훈련 데이터를 불러오는 함수
# csv 파일을 불러오는 함수입니다. 훈련 데이터들을 불러와서 새로운 변수에 담습니다.
# 데이터의 형태는 8x6 숫자이미지를 csv 파일화 한것입니다.
# 데이터의 형태 즉, 파일의 1행의 형태는 데이터 + 정답으로 이루어 지며, 1 x 52 가 됩니다.
# 정답은 이진수 bit 4개로 표현하였습니다.
def load_dataset():
    with open('./data_set2.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt = 48
    output_cnt = 4

    data = np.zeros([len(rows), input_cnt+output_cnt])
    output = np.zeros(4) 
    
    for n, row in enumerate(rows):
        data[n] = row

# 테스트 데이터를 불러오는 함수        
# csv 파일을 불러오는 함수입니다. 테스트 데이터들을 불러와서 새로운 변수에 담습니다.
# 데이터의 형태는 8x6 숫자이미지를 csv 파일화 한것입니다.
# 데이터의 형태 즉, 파일의 1행의 형태는 데이터 + 정답으로 이루어 지며, 1 x 52 가 됩니다.
# 정답은 이진수 bit 4개로 표현하였습니다.
def load_testset():
    with open('./train_set2.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global test, input_cnt, output_cnt
    input_cnt = 48
    output_cnt = 4

    test = np.zeros([len(rows), input_cnt+output_cnt])
    output = np.zeros(4) 
    
    for n, row in enumerate(rows):
        test[n] = row


In [16]:
# 설계한 신경망 모델을 초기화하는 함수입니다. 
# 신경망에 있어서 입력, 출력, 은닉계층 노드의 수에 대해 설정합니다.
# 또한, 은닉 계층 및 출력 계층에 대한 가중치 행렬을 생성합니다.
#weight_hidden, weight_output
def init_model_node():
    global weight_output, weight_hidden, input_cnt, output_cnt, hidden_cnt
    
    input_cnt = 48
    output_cnt = 4
    hidden_cnt = 10
    
    # 가중치 행렬 생성
    # input data의 수 48
    # 48개의 데이터가 한번에 들어가게 된다.
    # output data의 수 4 
    # 4개의 이진수 데이터로 구성된다.
    # weight_hidden => 은닉층 계층의 가중치 행렬
    # weight_output => 출력층 계층의 가중치 행렬
    weight_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    weight_output = alloc_param_pair([hidden_cnt, output_cnt])
    
    
# 가중치 행렬 생성을 위한 함수입니다.
# 난수를 이용해 가중치 행렬이 일정한 값으로 유지되지 않도록 설정해줍니다.
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

In [17]:
# 학습 데이터를 나누는 함수입니다.
# 불러온 파일에 대해서 데이터와 정답으로 나눕니다.
# 즉, 8x6 그림 파일에 대한 데이터와 그에 대한 정답 4bit로 나누는 과정입니다.
def get_train_data():
    global data, output_cnt
    train_data = data
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

# 테스트 데이터를 나누는 함수입니다.
# 불러온 파일에 대해서 데이터와 정답으로 나눕니다.
# 즉, 8x6 그림 파일에 대한 데이터와 그에 대한 정답 4bit로 나누는 과정입니다.
def get_test_data():
    global test, input_cnt, output_cnt
    test_data = test
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]


In [18]:
def init_model():
    print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
    init_model_node()

In [19]:
# 전방향으로 학습이 진행되도록 하는 함수입니다.
# forward_neuralnet_hidden1
# 
def forward_net(x):
    return forward_net_hidden(x)
    

In [20]:
# 싱글 레이어 퍼셉트론에서처럼 전방향으로 학습을 진행하는 함수입니다.
# hidden -> 입력층과 은닉계층 사이의 가중치 행렬 과 입력의 곱으로 형성됩니다.
# output -> 은닉계층과 출력계층 사이의 가중치 행렬 과 은닉계층의 출력의 곱으로 형성됩니다.
def forward_net_hidden(x):
    global data, weight_output, weight_hidden
    
    # forward porpagation 부분입니다.
   
    # 이 부분을 통해 입력은 은닉층을거쳐 출력층까지 전달됩니다.
    # 1 X 48 =>  1개의 INPUT DATA 
    # 48 X 10 => 1번째 계층의 가중치 행렬
    
    # hidden --> 10 X 4 =>  2번째 계층의 가중치 행렬 
    # output1 -->  X 4 => 최종 결과를 포함하고 있는 행렬
    hidden = sigmoid(np.matmul(x, weight_hidden['w']) + weight_hidden['b'])
    output = sigmoid(np.matmul(hidden, weight_output['w']) + weight_output['b'])
  
    return output, hidden

# 활성화 함수 - Sigmoid 함수
def sigmoid(x):
    return 1 / (1 +np.exp(-x))


In [21]:
# 출력 계층 부분에 대한 델타 값을 구하기 위한 함수입니다.
# PDF의 공식대로 델타를 구하도록 합니다.
# y X (1-y) X * (T - Y)
# T -> y
# y -> output
def output_delta(output, y):
    diff = y-output
    diff_y = 1 - output
    multiple_diff = diff * diff_y
    result = output * multiple_diff
    return result

# 은닉계층의 델타값에 대해 (Z X (1-Z))를 구하기 위한 부분입니다.
def output_hidden(hidden):
    return hidden * (1-hidden)


# 정확도를 계산하기 위한 
def evaluation(output, y):
    count_answer = 0
    arr_len = len(output)
    for index in range(arr_len) :
        if(np.array_equiv(output[index] ,y[index])):
            count_answer += 1
            
    final_acc = count_answer / arr_len
    return final_acc



In [22]:
# 전체적인 학습을 진행하는 함수입니다.
# forward, backward
# 전방향, 역방향 모두 학습을 진행시킵니다.
# 인자 x => 학습 데이터     인자 y => 학습 데이터에 대한 정답
# 데이터의 수만큼 반복문을 통해 학습을 진행합니다.
def run_train(x, y):
    # < Forward 부분 >
    global data, weight_output, weight_hidden
    
    # i -> 데이터의 갯수
    # x[n] -> n 번째 행의 데이터를 의미합니다.
    i = len(data)
    for n in range(i):

        # output -> forward 계산을 통한 output
        # hidden -> 은닉계층의 결과값
        # 1 x 4 , 1 x 10
        output, hidden = forward_net_hidden(x[n])

        # output delta 구하기
        # 출력층의 델타값에 대한 행렬은 1 x 4로 구성됩니다. 
        delta_output = output_delta(output, y[n])

        # 출력층으로 연결되는 가중치 수정
        # 은닉계층과 출력층 사이의 가중치 행렬을 수정하는 부분입니다.
        # 행렬 곱셈을 위해서 hidden 행렬의 모양을 바꾸기 위해서 reshape을 사용하였습니다.
        delta_output = delta_output.reshape((1,4))
        hidden = hidden.reshape((-1,1))
        
        # 가중치를 조정하기 위해서 PDF 의 공식을 구현하였습니다.
        delta = LEARNING_RATE * np.matmul(hidden, delta_output)

        # 은닉계층과 출력층 사이의 가중치 행렬을 수정해줍니다.
        new_hidden_weight = weight_output['w'] + delta
        weight_output['w'] = new_hidden_weight
        

# --------------------------------------------------------------------------        
        # < Backward 부분 >
        # hidden -> 10 x 1
        # new hidden -> 10 x 4
        # 입력층과 은닉계층 사이의 가중치 행렬을 수정하기 위해서 이전의 수정한 가중치 행렬을
        # deep copy 합니다.
        copy_hidden_weight = new_hidden_weight.copy()

        # 은닉 계층에 대한 델타를 생성하기 위한 구간
        # row =  output 계층과 연결되는 가중치 행렬 
        # PDF의 은닉계층의 델타 공식 중 시그마 안쪽의 곱셈 부분에 해당합니다.
        # 10 x 4
        index = 0
        for row in copy_hidden_weight:
            copy_hidden_weight[index] = row * delta_output 
            index += 1
      
        # 1 x 10의 새로운 값 형성
        # 시그마 부분 - 위에서 구했던 값을 이제 시그마를 통해 더해주는 구간입니다.
        copy_hidden_weight = copy_hidden_weight.sum(axis = 1)
      
        # z x (1-z) 부분
        # transpose() -> 이전의 행렬들과 곱셈을 위해서 전치 시켜줍니다.
        # 1 x 10 행렬이 형성됩니다.
        calculate_hidden = hidden.reshape(1,-1)
        calculate_hidden = output_hidden(calculate_hidden)
        
        # z * (1-z) * delta 
        # Learning rate X z * (1-z) * delta
        # 1 x 10 행렬이 형성됩니다.
        new_delta = calculate_hidden * copy_hidden_weight
        new_delta = LEARNING_RATE * new_delta

        # Learning rate X delta * z(k)
        # z(k)를 곱하기 위해서 x[n]을 전치 시켜줍니다.
        trans_x = x[n].copy()
        trans_x = trans_x.reshape(-1,1)

        # 새로운 가중치 행렬을 위한 변화량을 최종적으로 구성합니다.
        delta_weight =  trans_x * new_delta 
        
        # 새로운 가중치가 형성되었습니다.
        final_weight = weight_hidden['w'] + delta_weight
        weight_hidden['w'] = final_weight
    
    result, result_hidden = forward_net(x)
    result = output > 0.5
    result = result.astype(int) 
    
    accuracy = evaluation(result, y)
    return accuracy  
        
# 학습을 완료한 후 테스트 데이터를 테스트하는 함수입니다.
def run_test(test_for_x, test_for_y):
    global test, data, weight_output, weight_hidden

    output, hidden = forward_net(test_for_x)
    output = output > 0.5
    output = output.astype(int) 
    accuracy = evaluation(output, test_for_y)
    return accuracy


In [23]:
def train_and_test(epoch_count):
    global data, test, weight_output, weight_hidden
    
    load_dataset()
    load_testset()
    test_x, test_y = get_test_data()
    train_x, train_y = get_train_data()
 
    for epoch in range(epoch_count):
        acc = run_train(train_x, train_y)     
        print('Epoch {}: accuracy={:5.3f}'.format(epoch+1, acc))
  
    last_accurancy = run_test(test_x, test_y)
    print('\nTest: accuracy = {:5.3f}'.format(last_accurancy))

In [24]:
init_model()
train_and_test(100)

은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 1: accuracy=0.750
Epoch 2: accuracy=0.750
Epoch 3: accuracy=0.750
Epoch 4: accuracy=0.750
Epoch 5: accuracy=0.750
Epoch 6: accuracy=0.500
Epoch 7: accuracy=0.500
Epoch 8: accuracy=0.500
Epoch 9: accuracy=0.500
Epoch 10: accuracy=0.500
Epoch 11: accuracy=0.500
Epoch 12: accuracy=0.500
Epoch 13: accuracy=0.500
Epoch 14: accuracy=0.500
Epoch 15: accuracy=0.500
Epoch 16: accuracy=0.500
Epoch 17: accuracy=0.500
Epoch 18: accuracy=0.500
Epoch 19: accuracy=0.500
Epoch 20: accuracy=0.500
Epoch 21: accuracy=0.500
Epoch 22: accuracy=0.500
Epoch 23: accuracy=0.500
Epoch 24: accuracy=0.500
Epoch 25: accuracy=0.500
Epoch 26: accuracy=0.500
Epoch 27: accuracy=0.500
Epoch 28: accuracy=0.500
Epoch 29: accuracy=0.500
Epoch 30: accuracy=0.500
Epoch 31: accuracy=0.500
Epoch 32: accuracy=0.500
Epoch 33: accuracy=0.500
Epoch 34: accuracy=0.500
Epoch 35: accuracy=0.500
Epoch 36: accuracy=0.500
Epoch 37: accuracy=0.500
Epoch 38: accuracy=0.500
Epoch 39: accuracy=0.500
Epo